In [37]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section c.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(446425, 19)


,hhcode,province,district,region,psu,hhno,idc,sec,scq01,scq02,scq03,scq04,scq05,scq06,scq07,scq08a,scq08b,scq09a,scq09b
221277,2.082001e+09,2,208,2,20820013,8,7,C,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,4.0,12.0
44863,1.092003e+09,1,109,2,10920032,12,2,C,1.0,1.0,1,12.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
149706,1.282001e+09,1,128,2,12820006,1,8,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
18499,1.042001e+09,1,104,2,10420009,10,7,C,2.0,1.0,2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
159270,1.302002e+09,1,130,2,13020016,16,4,C,1.0,1.0,1,7.0,1.0,8.0,1.0,1.0,NaN,NaN,NaN


In [38]:
# Create a yes/no column for "ever admitted"
C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scq04"]
C["currently_enrrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

In [39]:
# Reduce to the questions of interest (4 questions):

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [40]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.001100e+09,1,2,NaN,NaN,6.0,1,1
1,1.001100e+09,2,2,NaN,NaN,5.0,1,1
2,1.001100e+09,3,2,NaN,NaN,NaN,1,1
3,1.001100e+09,1,1,5.0,2.0,NaN,1,1
4,1.001100e+09,2,2,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...
446420,4.262002e+09,3,2,NaN,NaN,6.0,2,4
446421,4.262002e+09,4,2,NaN,NaN,4.0,2,4
446422,4.262002e+09,5,2,NaN,NaN,4.0,2,4
446423,4.262002e+09,6,2,NaN,NaN,6.0,2,4


In [41]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section b.dta"
)
B.sample(5)

,hhcode,province,district,region,psu,section,idc,sbq01,sbq02,sbq03,sbq41,sbq42,sbq43,age,sbq05
32467,1.071001e+09,punjab,khushab,urban,10710014,B,3,male,present,son/daughter,1989,12,17,17,unmarried
113270,1.191002e+09,punjab,kasur,urban,11910015,B,5,female,present,son/daughter,1995,1,8,11,unmarried
43853,1.091202e+09,punjab,faisalabad,urban,10912023,B,1,male,present,head of household,1969,0,0,36,married
401640,3.202000e+09,nwfp,haripur,rural,32020002,B,3,female,present,son/daughter,1988,0,0,18,unmarried
475277,4.192001e+09,balochistan,lorali,rural,41920007,B,3,male,present,son/daughter,1997,0,0,9,unmarried


In [42]:
# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [43]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [44]:
# MERGE
years06_07 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years06_07._merge.value_counts() #(ALL GOOD)

In [45]:
# Select people in school enrollment age (I would say 4-10 years old)
years06_07 = years06_07.loc[(years06_07["age"] >= 4)&(years06_07["age"] <= 15)]

In [46]:
years06_07.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    66049
2    42560
3    34792
4    29827
Name: province, dtype: int64

In [47]:
# Lets see which hh codes we have:
years06_07["hhcode"] = years06_07["hhcode"].astype("str")
years06_07['subprovince code']= years06_07['hhcode'].apply(lambda x: x[0:4])
years06_07["subprovince code"] = years06_07["subprovince code"].astype("int")

In [48]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0607_v1.csv"
)
strat_name.sample(5)

,province,subprovince,subprovince code long
36,Punjab,Sahiwal,12320
14,Punjab,Attock,10120
34,Punjab,Sheikhupura,12120
92,Balochistan,Sibbi,40520
116,Punjab,Islamabad,10020


In [49]:
strat_name['subprovince code long']= strat_name['subprovince code long'].astype("str")
strat_name['subprovince code']= strat_name['subprovince code long'].apply(lambda x: x[0:4])
strat_name['subprovince code'] = strat_name['subprovince code'].astype("int")
strat_name.sample(5)

,province,subprovince,subprovince code long,subprovince code
11,Sindh,Karachi,21611,2161
70,NWFP,Bonair,30720,3072
30,Punjab,Narowal,11720,1172
17,Punjab,Chakwal,10420,1042
54,Sindh,Shikarpur,20720,2072


In [50]:
# MERGE
years06_07_ = pd.merge(years06_07, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years06_07_._merge.value_counts()

both          144401
left_only          0
right_only         0
Name: _merge, dtype: int64

In [51]:
years06_07_

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province_x,subprovince code,province_y,subprovince,subprovince code long,_merge
0,1001100103.0,5,15,female,unmarried,1,10.0,2.0,5.0,1,1,1001,Punjab,Islamabad,10011,both
1,1001100103.0,6,13,male,unmarried,1,6.0,1.0,NaN,1,1,1001,Punjab,Islamabad,10011,both
2,1001100104.0,4,15,female,unmarried,1,5.0,2.0,5.0,1,1,1001,Punjab,Islamabad,10011,both
3,1001100104.0,5,13,female,unmarried,1,5.0,2.0,5.0,1,1,1001,Punjab,Islamabad,10011,both
4,1001100104.0,6,11,female,unmarried,1,5.0,2.0,5.0,1,1,1001,Punjab,Islamabad,10011,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144396,4262002013.0,5,12,female,unmarried,2,NaN,NaN,13.0,2,4,4262,Balochistan,Bolan/kachhi,42620,both
144397,4262002013.0,6,8,female,unmarried,2,NaN,NaN,13.0,2,4,4262,Balochistan,Bolan/kachhi,42620,both
144398,4262002014.0,5,15,female,unmarried,2,NaN,NaN,13.0,2,4,4262,Balochistan,Bolan/kachhi,42620,both
144399,4262002014.0,6,11,male,unmarried,2,NaN,NaN,4.0,2,4,4262,Balochistan,Bolan/kachhi,42620,both


In [52]:
years06_07_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years06_07_.csv', index_label="id")